In [1]:
import os
from typing import Union

from transformers import AutoTokenizer
import transformers
import torch

In [3]:
!nvidia-smi

Sat May  4 22:29:16 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 530.30.02              Driver Version: 530.30.02    CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                  Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3090         Off| 00000000:3B:00.0 Off |                  N/A |
| 40%   51C    P2              115W / 350W|  21206MiB / 24576MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [12]:
os.environ['CUDA_VISIBLE_DEVICES'] = '3'

In [4]:
torch.cuda.is_available()

True

In [10]:
torch.cuda.device_count()

1

In [5]:
torch.cuda.current_device()

0

In [12]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3090'

In [4]:
device = torch.device('cuda:3' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda:3


In [5]:
from vllm import LLM, SamplingParams

In [6]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = SamplingParams(temperature=0, top_p=0.95)

In [9]:
model = "../../llama/llama-2-7b-chat-hf"

In [13]:
!export CUDA_VISIBLE_DEVICES=3

In [15]:
llm = LLM(model=model)

INFO 03-28 00:14:45 llm_engine.py:87] Initializing an LLM engine with config: model='../../llama/llama-2-7b-chat-hf', tokenizer='../../llama/llama-2-7b-chat-hf', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=2048, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=0)
INFO 03-28 00:15:00 llm_engine.py:357] # GPU blocks: 1030, # CPU blocks: 512
INFO 03-28 00:15:03 model_runner.py:684] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 03-28 00:15:03 model_runner.py:688] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can al

In [17]:
outputs = llm.generate(prompts, sampling_params)

Processed prompts: 100%|██████████| 4/4 [00:00<00:00, 12.51it/s]


In [18]:
# Print the outputs.
for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

Prompt: 'Hello, my name is', Generated text: " Sherry and I'm a 35-year-old woman from"
Prompt: 'The president of the United States is', Generated text: ' a member of Congress, which means that he or she is subject to the same'
Prompt: 'The capital of France is', Generated text: ' Paris. This is a fact that is well known and widely accepted. However,'
Prompt: 'The future of AI is', Generated text: ' likely to be shaped by a combination of technological advancements, soci'


In [7]:
def load_llama_model(path: Union[str, os.PathLike]):
    tokenizer = AutoTokenizer.from_pretrained(path)
    pipeline = transformers.pipeline(
        "text-generation",
        model=model,
        torch_dtype=torch.float16,
        device_map="auto",
    )
    return pipeline, tokenizer

In [34]:
pipeline, tokenizer = load_llama_model(model)

OSError: ../../llama/llama-2-7b-chat does not appear to have a file named config.json. Checkout 'https://huggingface.co/../../llama/llama-2-7b-chat/None' for available files.

In [32]:
os.getcwd()

'/home/adiel/code/notebooks'